### Сделайте краткий обзор любой архитектуры для object detection. Проведите анализ: чем отличается выбранная вами архитектура нейронной сети от других? В чём плюсы и минусы данной архитектуры? Какие могут возникнуть трудности при применении этой архитектуры на практике?

Сеть Mask R-CNN была разработана в 2017 году и позволяет решать задачи instance segmentation (сегментации обёектов на изображениии). 
В частности, сеть с архитектурой Mask R-CNN позволяет выделять на фотографиях контуры («маски») экземпляров разных объектов, 
даже если таких экземпляров несколько, они имеют различный размер и частично перекрываются. 
Сеть так же способна к распознаванию поз людей на изображении.

Концепции, лежащие в основе Mask R-CNN прошли поэтапное развитие через архитектуры нескольких промежуточных нейросетей, решавших задачи Classification, Semantic segmentation, Object detection, Instance segmentation. Поэтому на примере этой сети рассмотрим 
все сильные стороны и недостатки, которые сеть унаследовала от своих предшественников.


Mask R-CNN многослойная двухстадийная сеть. Имеет верточный слой, который позволяет объединять значения расположенных рядом пикселей и выделять более обобщённые признаки изображения. Для этого по изображению последовательно проходят ядром (kernel). Второй способ получения
обобщённой характеристики группы пикселей — выбрать один пиксель по заданному правилу, например — максимальный. Именно это и делает слой MaxPooling.

За основу взята сеть CNN, которая хорошо решает задачи классификации. Использовалась готовая архитектура — CaffeNet (AlexNet), способная 
делать классификацию на 1000 классов. R-CNN разрабатывалась для детектирования объектов меньшего количества классов (20 - 200), поэтому последний классификационный слой CaffeNet был заменён на слой с N+1 выходами (с дополнительным классом для фона).

Из недостатков - CaffeNet принимает на вход изображения фиксированного размера 227х227 пикселей, поэтому перед подачей изображений на вход сети их приходилось модифицировать и масштабировать под нужный размер.

Процедуру детектирования объектов сетью R-CNN можно разделить на следующие шаги:
1 Выделение регионов-кандидатов при помощи Selective Search.
2 Преобразование региона в размер, принимаемый CNN CaffeNet.
3 Получение при помощи CNN 4096-размерного вектора признаков.
4 Проведение бинарных классификаций каждого вектора признаков при помощи N линейных SVM.
5 Линейная регрессия параметров рамки региона для более точного охвата объекта

Несмотря на хорошие результаты, производительность R-CNN была невысока, особенно для более глубоких сетей. Кроме того, требовалось сохранение на диск большого количества признаков. Процедура "дорогая" с точки зрения размера хранилища.

Чтобы увеличить производительность в Fast R-CNN предложили ускорить процесс за счёт следующих модификаций:
- Пропускать через CNN не каждый из 2000 регионов-кандидатов по отдельности, а всё изображение целиком. Предложенные регионы потом накладываются на полученную общую карту признаков;
- Вместо независимого обучения трёх моделей (CNN, SVM, bbox regressor) совместить все процедуры тренировки в одну.

Дальнейшая практика показала, что после улучшений, сделанных в Fast R-CNN, самым узким местом нейросети оказался 
механизм генерации регионов-кандидатов. Было предложено предложили вычислять регионы не по изначальному изображению, 
по карте признаков, полученных из CNN. Для этого был добавлен модуль под названием Region Proposal Network (RPN) и сеть 
получила название Faster R-CNN.

В рамках RPN по извлечённым CNN признакам проходят нейросетью с небольшим (3х3) окном. 
Полученные с её помощью значения передаются в два параллельных полносвязанных слоя.
Оба слоя RPN выдают только предложения для регионов-кандидатов. Те из них, которые имеют высокую вероятность содержания какого-либо объекта, передаются дальше в модуль детектирования объектов и уточнения охватывающей рамки, который по-прежнему реализован как 
Fast R-CNN.

Mask R-CNN развивает архитектуру Faster R-CNN путём добавления ещё одной ветки, которая предсказывает положение маски, покрывающей найденный объект, и, таким образом решает уже задачу instance segmentation. 
Условно архитектура разделяется на CNN-сеть вычисляющую признаки изображения, называемую backbone, и head — объединение частей, отвечающих за предсказание охватывающей рамки, классификацию объекта и определение его маски.

Помимо высоких результатов в задачах instance segmentation и object detection, Mask R-CNN оказалась пригодной для определения поз людей на фотографии с помощью определения опорных точек.

Для определения опорных точек нейросеть обучают таким образом, чтобы она выдавала маски, в которых только один пиксель 
(та самая точка) имел значение 1, а остальные — 0 (one-hot mask).

Не смотря на то, что сеть уже не "молодая", она до сих пор используется и имеет потенциал для увеличения 
производительности и точности. Однако она всё же уступает в скорости "одностадийным" сетям (YOLO, SSD, в которых результат выдается за "один проход". 
В итоге нельзя сказать, что какая-то сеть лучше другой. Совершенствуются алгоритмы, жизнь и бизнес ставят новые задачи.
Необходимо под каждую конкретную задачу подбирать сеть, которая с такими задачами справляется лучше других.